<a href="https://colab.research.google.com/github/AndresMontesDeOca/Laboratorio3/blob/main/Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Experiments

## Libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# from neuralprophet import NeuralProphet
import tensorflow as tf


import warnings
# warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore')

# Ajustar la opción para mostrar más filas
# pd.set_option('display.max_rows', None)

# Si también quieres mostrar más columnas
# pd.set_option('display.max_columns', None)


# Vamos a suprimir la notacion cientifica
pd.set_option("display.float_format", lambda x:"%.2f" %x)


## Carga Datos

In [ ]:
# Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

################################# Datasets ###################################
# # Ventas
id = "158aOjqxaNO8l97yA6VWJkek_15YVLMhs"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('sell-in.txt')
data_ventas = pd.read_csv("sell-in.txt", sep="\t")
data_ventas['periodo'] = pd.to_datetime(data_ventas['periodo'], format='%Y%m')
data = data_ventas.copy()

# # Productos
id = "15JS_k86LS0sgJXma7BOVXWlyNcMwxdhE"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_productos.txt')
data_productos = pd.read_csv("tb_productos.txt", sep="\t")

# # Stocks
id = "15EV-8f_U7onpA1AcTxxXeD-z8yVR4fQu"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_stocks.txt')
data_stocks = pd.read_csv("tb_stocks.txt", sep="\t")
data_stocks['periodo'] = pd.to_datetime(data_stocks['periodo'], format='%Y%m')

# # Productos a predecir
id = "15LjADctFVwjzQFJvfJGFTEdgZx9xCoId"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('productos_a_predecir.txt')
data_productos_a_predecir = pd.read_csv("productos_a_predecir.txt", sep="\t")



## Group Data

In [ ]:
import pandas as pd

def group_data(data_all, data_filter):
    # Filtrar el DataFrame 'data_all' para que solo contenga los 'product_id' presentes en 'data_filter'
    data_filtered = data_all[data_all['product_id'].isin(data_filter['product_id'])]

    # Agrupa los datos por 'product_id' y 'periodo', y calcula la suma de 'tn'
    grouped_data = data_filtered.groupby(['product_id', 'periodo']).sum().reset_index()

    # Crea un DataFrame pivoteado donde las filas son las fechas y las columnas son los product_id
    pivot_data = grouped_data.pivot(index='periodo', columns='product_id', values='tn')

    # Rellena los NaN
    pivot_data = pivot_data.fillna(0)

    # Asegúrate de que los nombres de las columnas sean strings
    pivot_data.columns = pivot_data.columns.astype(str)

    # Restablece el índice para asegurarse de que 'product_id' no sea un índice compuesto
    pivot_data.columns.name = None

    return pivot_data

data_new = group_data(data, data_productos_a_predecir)

In [ ]:
# Filtrar el DataFrame 'data' para que solo contenga los 'product_id' presentes en 'data_productos_a_predecir'
data_filtered = data[data['product_id'].isin(data_productos_a_predecir['product_id'])]


# Agrupa los datos por 'product_id' y 'periodo', y calcula la suma de 'tn'
grouped_data = data_filtered.groupby(['product_id', 'periodo']).sum().reset_index()

# Crea un DataFrame pivoteado donde las filas son las fechas y las columnas son los product_id
pivot_data = grouped_data.pivot(index='periodo', columns='product_id', values='tn')

# Rellena los NaN
pivot_data = pivot_data.fillna(0)

# Asegúrate de que los nombres de las columnas sean strings
pivot_data.columns = pivot_data.columns.astype(str)

# Restablece el índice para asegurarse de que 'product_id' no sea un índice compuesto
pivot_data.columns.name = None

data_2019 = pivot_data.loc['2019']

In [ ]:
data_2019

,20001,20002,20003,20004,20005,20006,20007,20008,20009,20010,...,21248,21252,21256,21259,21262,21263,21265,21266,21267,21276
periodo,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1275.77,1266.79,964.77,511.34,363.58,578.74,377.67,543.28,465.48,370.76,...,0.03,0.00,0.04,0.04,0.03,0.04,0.00,0.00,0.00,0.00
2019-02-01,1259.09,1043.01,758.33,441.70,409.90,480.00,368.80,476.99,366.73,337.76,...,0.06,0.00,0.05,0.05,0.06,0.06,0.00,0.00,0.00,0.00
2019-03-01,1470.66,1083.63,638.04,619.77,488.21,502.44,576.23,454.57,525.47,522.88,...,0.05,0.38,0.06,0.07,0.06,0.07,0.02,0.02,0.22,0.12
2019-04-01,1647.64,1287.62,565.34,466.71,625.00,835.48,511.55,403.69,391.28,446.72,...,0.03,0.23,0.02,0.02,0.01,0.02,0.36,0.36,0.12,0.10
2019-05-01,1629.78,1034.99,590.13,603.31,897.26,527.69,394.68,486.37,578.49,408.42,...,0.02,0.47,0.02,0.02,0.01,0.02,0.18,0.18,0.24,0.09
2019-06-01,1109.94,928.36,662.39,667.19,876.40,458.04,536.14,567.42,610.40,600.25,...,0.01,0.08,0.01,0.01,0.01,0.01,0.05,0.05,0.06,0.04
2019-07-01,1678.99,1066.45,715.20,521.72,745.75,343.11,573.37,524.05,716.08,463.92,...,0.02,0.03,0.03,0.05,0.04,0.03,0.03,0.06,0.07,0.00
2019-08-01,1261.35,813.78,635.60,482.13,536.67,262.74,307.83,233.01,520.42,199.86,...,0.02,0.13,0.03,0.03,0.02,0.03,0.02,0.01,0.04,0.01
2019-09-01,1660.01,1090.19,967.77,786.17,879.53,409.96,369.75,330.56,558.46,524.95,...,0.01,0.14,0.00,0.01,0.02,0.01,0.02,0.03,0.02,0.02


## Normalize Data

In [ ]:
import pandas as pd

def normalize_data(df, normalization="MinMax"):
    """
    Normaliza cada serie de tiempo (columna) de manera individual usando MinMax o Zscore.

    Args:
        df (pd.DataFrame): DataFrame con series de tiempo de distintos productos, cada columna es un producto.
        normalization (str): Tipo de normalización a aplicar. Opciones: "MinMax" o "Zscore". Default es "MinMax".

    Returns:
        normalized_df (pd.DataFrame): DataFrame con las series normalizadas.
        normalization_params (dict): Diccionario con los parámetros necesarios para desnormalizar cada columna.
            - Para "MinMax": valores min y max de cada columna.
            - Para "Zscore": valores mean y std de cada columna.
    """
    normalization_params = {}
    normalized_df = pd.DataFrame()

    for column in df.columns:
        if normalization == "MinMax":
            min_value = df[column].min()
            max_value = df[column].max()
            normalization_params[column] = {"min": min_value, "max": max_value}
            normalized_df[column] = (df[column] - min_value) / (max_value - min_value)

        elif normalization == "ZScore":
            mean_value = df[column].mean()
            std_value = df[column].std()
            normalization_params[column] = {"mean": mean_value, "std": std_value}
            normalized_df[column] = (df[column] - mean_value) / std_value

        else:
            raise ValueError("Invalid normalization method. Choose 'MinMax' or 'Zscore'.")

    return normalized_df, normalization_params


In [ ]:
data

,20001,20002,20003,20004,20005,20006,20007,20008,20009,20010,...,21248,21252,21256,21259,21262,21263,21265,21266,21267,21276
periodo,,,,,,,,,,,,,,,,,,,,,
2019-01-01,0.29,0.39,0.77,0.11,0.00,0.55,0.26,0.94,0.28,0.43,...,0.38,0.00,0.62,0.59,0.30,0.60,0.00,0.00,0.00,0.00
2019-02-01,0.26,0.20,0.37,0.00,0.07,0.38,0.23,0.76,0.00,0.34,...,1.00,0.00,0.85,0.73,0.89,0.87,0.00,0.00,0.00,0.00
2019-03-01,0.63,0.23,0.14,0.29,0.20,0.42,1.00,0.70,0.45,0.81,...,0.84,0.81,1.00,1.00,1.00,1.00,0.04,0.05,0.88,1.00
2019-04-01,0.94,0.41,0.00,0.04,0.41,1.00,0.76,0.56,0.07,0.62,...,0.41,0.50,0.26,0.25,0.05,0.15,1.00,1.00,0.50,0.83
2019-05-01,0.91,0.19,0.05,0.26,0.84,0.46,0.32,0.78,0.61,0.52,...,0.11,1.00,0.21,0.23,0.00,0.22,0.48,0.49,1.00,0.76
2019-06-01,0.00,0.10,0.19,0.36,0.81,0.34,0.85,1.00,0.70,1.00,...,0.00,0.17,0.10,0.07,0.03,0.00,0.14,0.14,0.24,0.33
2019-07-01,1.00,0.22,0.29,0.13,0.60,0.14,0.99,0.88,1.00,0.66,...,0.22,0.06,0.44,0.66,0.54,0.42,0.08,0.16,0.30,0.02
2019-08-01,0.27,0.00,0.14,0.06,0.27,0.00,0.00,0.10,0.44,0.00,...,0.22,0.28,0.46,0.41,0.22,0.42,0.04,0.04,0.17,0.10
2019-09-01,0.97,0.24,0.78,0.55,0.81,0.26,0.23,0.36,0.55,0.81,...,0.08,0.31,0.00,0.00,0.08,0.02,0.05,0.08,0.07,0.15


## Split Data

In [ ]:
def split_data_all(data):
  data_train = data.loc['2017':'2018']
  data_valid = data.loc['2019']
  return data_train, data_valid

data_train, data_valid = split_data_all(data_all_norm)

(24, 780)
(12, 780)


## Wrangle Data

In [ ]:
def windowed_dataset(sequence, data_split, window_size, batch_size, n_future, shuffle_buffer=1000, seed=None):
    """Generates dataset windows for multi-step forecasting in a multivariable context.

    Args:
      sequence (array-like): Contains the values of the time series, where each element is an array of feature values.
      data_split (str): Specifies if the dataset is for training or validation/test.
      window_size (int): The number of time steps to include in the feature.
      batch_size (int): The batch size.
      n_future (int): The number of future steps to predict.
      shuffle_buffer (int): Buffer size to use for the shuffle method.
      seed (int, optional): Random seed for reproducibility.

    Returns:
      tf.data.Dataset: TF Dataset containing time windows.
    """

    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(sequence)

    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + n_future, shift=1, drop_remainder=True)

    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + n_future))

    # Create tuples with features and labels
    dataset = dataset.map(lambda window: (window[:window_size], window[window_size:]))

    if data_split == 'train':
        # Shuffle the training data to improve generalization
        dataset = dataset.shuffle(shuffle_buffer, seed=seed)
    else:
        # Cache the validation/test data for improved performance
        dataset = dataset.cache()

    # Create batches of windows and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset


# Pipeline

In [ ]:
# data
# data_productos
# data_stocks
# data_productos_a_predecir
# # if __name__ == '__main__':
window_size = 12
n_future = 2
batch_size = 64
normalization = 'ZScore'

data_all = group_data(data, data_productos_a_predecir)
data_all_norm, data_all_norm_params = normalize_data(data_all, normalization=normalization)
data_all_norm['20001'].describe()
# data_train, data_valid = split_data_all(data_all_norm)


count   36.00
mean     0.00
std      1.00
min     -2.01
25%     -0.56
50%      0.07
75%      0.56
max      3.01
Name: 20001, dtype: float64